In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

## Prioritas Scrapping

In [2]:
path_of_the_directory= '/work/Prioritas'

In [3]:
prioritas2024 = []
prioritas2019 = []
for filename in os.listdir(path_of_the_directory):
    fname = path_of_the_directory + '/' + filename
    f = open(fname, 'r')
    soup= BeautifulSoup(f.read(), 'html.parser')
    items = soup.find_all('tr', {'role':'row'})[1:-1]
    year = filename[:4]
    for item in items:
        name = item.find('a').text.strip()
        info = item.find_all('td')
        date = info[4].text.strip()
        sponsor = info[9].text.strip()
        status = info[5].text.strip()
        data = [name, date, sponsor, status, year]
        category = info[7].text
        if category == 'Prolegnas 2015-2019':
            prioritas2019.append(data)
        else:
            prioritas2024.append(data) 

In [4]:
print(prioritas2019)

[['RUU tentang Perubahan atas Undang-Undang Nomor 32 Tahun 2002 tentang Penyiaran', '10 Mar 2020', 'DPR', 'Harmonisasi', '2015'], ['RUU tentang Radio Televisi Republik Indonesia', '07 Feb 2018', 'DPR', 'Penyusunan', '2015'], ['RUU tentang Perubahan atas Undang - Undang Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik (ITE)', '08 Des 2016', 'PEMERINTAH', 'Selesai', '2015'], ['RUU tentang Wawasan Nusantara', '24 Okt 2016', 'DPD', 'Pembahasan', '2015'], ['RUU tentang Pertanahan', '09 Sep 2019', 'DPR', 'Pembahasan', '2015'], ['RUU tentang Perubahan atas Undang - Undang Nomor 1 Tahun 2015 tentang Penetapan Peraturan Pemerintah Pengganti UU Nomor 1 Tahun 2014 tentang Pemilihan Gubernur, Bupati, dan Walikota Menjadi UU', '08 Apr 2016', 'DPR', 'Selesai', '2015'], ['RUU tentang Perubahan Kedua Atas UU Nomor 23 Tahun 2014 tentang Pemerintahan Daerah', '23 Agust 2022', 'DPR', 'Selesai', '2015'], ['RUU tentang Peningkatan Pendapatan Asli Daerah', '23 Agust 2022', 'DPR', 'Terdaftar', 

In [5]:
df2019_prioritas_raw = pd.DataFrame(prioritas2019, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Tahun Prioritas'])
df2019_prioritas_raw.head()

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Tahun Prioritas
0,RUU tentang Perubahan atas Undang-Undang Nomor...,10 Mar 2020,DPR,Harmonisasi,2015
1,RUU tentang Radio Televisi Republik Indonesia,07 Feb 2018,DPR,Penyusunan,2015
2,RUU tentang Perubahan atas Undang - Undang Nom...,08 Des 2016,PEMERINTAH,Selesai,2015
3,RUU tentang Wawasan Nusantara,24 Okt 2016,DPD,Pembahasan,2015
4,RUU tentang Pertanahan,09 Sep 2019,DPR,Pembahasan,2015


In [6]:
#remove duplicate
df2019_prioritas = df2019_prioritas_raw.copy()
df2019_prioritas = df2019_prioritas.sort_values('Tahun Prioritas', ascending=False)
df2019_prioritas = df2019_prioritas.drop_duplicates(subset='Judul Ruu', keep='first')
print(len(df2019_prioritas))

103


In [7]:
df2024_prioritas_raw = pd.DataFrame(prioritas2024, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Tahun Prioritas'])
df2024_prioritas_raw

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Tahun Prioritas
0,RUUtentang Perubahan atas Undang-Undang Nomor ...,20 Jan 2021,DPR,Harmonisasi,2020
1,RUU tentang Kitab Undang-Undang Hukum Pidana,06 Jul 2022,"DPR, PEMERINTAH",Pembahasan,2020
2,RUU tentang Perubahan atas Undang-Undang Nomor...,07 Jul 2022,"DPR, PEMERINTAH",Pembahasan,2020
3,RUU tentang Jabatan Hakim,23 Agust 2022,DPR,Terdaftar,2020
4,RUU tentang Perubahan atas Undang-Undang Nomor...,07 Des 2021,"DPR, PEMERINTAH",Selesai,2020
...,...,...,...,...,...
109,RUU tentang Wabah,23 Agust 2022,PEMERINTAH,Terdaftar,2022
110,RUU tentang Badan Pembinaan Ideologi Pancasila,12 Mei 2020,PEMERINTAH,Penetapan Usul,2022
111,RUU tentang Desain Industri,23 Agust 2022,PEMERINTAH,Terdaftar,2022
112,RUU tentang Daerah Kepulauan,23 Agust 2022,DPD,Pembahasan,2022


In [8]:
#remove duplicate
df2024_prioritas = df2024_prioritas_raw.copy()
df2024_prioritas = df2024_prioritas.sort_values('Tahun Prioritas', ascending=False)
df2024_prioritas = df2024_prioritas.drop_duplicates(subset='Judul Ruu', keep='first')
print(len(df2024_prioritas))

66


## Prolegnas Scrapping

In [9]:
def connectDPR(url):
    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    req = requests.get(url, headers=headers)
    print(req)
    return BeautifulSoup(req.text, 'html.parser')

### Test

In [10]:
soup = connectDPR('https://www.dpr.go.id/uu/detail/id/239')
head = soup.find('div', {'class':'card-body'})

name = head.find('h3', {'class':'title-detail'}).text
date = head.find('li').text[17:]
sponsor = head.find('span', {'class':'label label-dpr'}).text

print(name, date, sponsor)

<Response [200]>
RUU tentang Perubahan atas Undang-Undang Nomor 32 Tahun 2002 tentang Penyiaran 17 Desember 2019 DPR


### Create a dataframe of prolegnas from main page

In [11]:
url24 = 'https://www.dpr.go.id/uu/prolegnas-long-list'
soup = connectDPR(url24)
head = soup.find('tbody')
items = head.find_all('tr', 'role' == 'row')

<Response [200]>


In [12]:
prioritas2024_list = df2024_prioritas['Judul Ruu'].tolist()
prioritas2019_list = df2019_prioritas['Judul Ruu'].tolist()
print('RUU tentang Jabatan Hakim' in prioritas2024_list)

True


In [13]:
data2024 = []
data2019 = []
for item in items:
    name = item.find('a').text.strip()
    info = item.find_all('td')
    date = info[4].text.strip()
    sponsor = info[9].text.strip()
    status = info[5].text.strip()
    print(name)
    print(status)
    category = info[7].text.strip()
    prioritas = 'Tidak'
    if category == 'Prolegnas 2015-2019':
        if name in prioritas2019_list:
            prioritas = 'Ya'
        data = [name, date, sponsor, status, prioritas]
        data2019.append(data)
    else:
        if name in prioritas2019_list:
            prioritas = 'Ya'
        data = [name, date, sponsor, status, prioritas]
        data2024.append(data) 

RUU tentang Perubahan atas Undang-Undang Nomor 24 Tahun 2000 tentang Perjanjian Internasional
Terdaftar
RUU tentang Keamanan dan Ketahanan Siber
Terdaftar
RUU tentang Perubahan atas Undang-Undang Nomor 37 Tahun 1999 tentang Hubungan Luar Negeri
Terdaftar
RUU tentang Perubahan atas Undang-Undang Nomor 32 Tahun 2002 tentang Penyiaran
Penyusunan
RUU tentang Perubahan atas Undang - Undang Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik (ITE)
Selesai
RUU tentang Radio Televisi Republik Indonesia
Terdaftar
RUU tentang Persandian
Terdaftar
RUU tentang Keamanan dan Ketahanan Siber
Penyusunan
RUU tentang Keamanan Laut
Terdaftar
RUU tentang Rahasia Negara
Terdaftar
RUU tentang Perubahan atas Undang-Undang Nomor 3 Tahun 2002 tentang Pertahanan
Terdaftar
RUU tentang Keamanan Nasional
Terdaftar
RUU tentang Tugas Perbantuan Militer
Terdaftar
RUU tentang Perubahan atas Undang-Undang Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik
Terdaftar
RUU tentang Radio Televisi Repu

In [14]:
df2019_main = pd.DataFrame(data2019, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Prioritas'])
df2019_main.head()

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Prioritas
0,RUU tentang Perubahan atas Undang-Undang Nomor...,27 Agust 2022,"DPR, PEMERINTAH",Terdaftar,Tidak
1,RUU tentang Perubahan atas Undang-Undang Nomor...,27 Agust 2022,"DPR, PEMERINTAH",Terdaftar,Tidak
2,RUU tentang Perubahan atas Undang - Undang Nom...,08 Des 2016,PEMERINTAH,Selesai,Ya
3,RUU tentang Persandian,27 Agust 2022,"DPR, PEMERINTAH",Terdaftar,Tidak
4,RUU tentang Keamanan dan Ketahanan Siber,28 Mei 2019,DPR,Penyusunan,Ya


In [15]:
df2024_main = pd.DataFrame(data2024, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Prioritas'])
df2024_main.head()

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Prioritas
0,RUU tentang Keamanan dan Ketahanan Siber,27 Agust 2022,DPR,Terdaftar,Ya
1,RUU tentang Perubahan atas Undang-Undang Nomor...,27 Agust 2022,DPR,Penyusunan,Ya
2,RUU tentang Radio Televisi Republik Indonesia,27 Agust 2022,DPR,Terdaftar,Ya
3,RUU tentang Keamanan Laut,27 Agust 2022,DPR,Terdaftar,Tidak
4,RUU tentang Perubahan atas Undang-Undang Nomor...,27 Agust 2022,DPR,Terdaftar,Tidak


In [16]:
print(len(df2019_main))
print(len(df2024_main))

190
256


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=45b28f99-d503-49a3-bd9e-f98821f9fde8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>